In [1]:
import os
from datetime import datetime
from dateutil.parser import parse
import time

import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint
from tqdm import tqdm

import seaborn as sns
sns.set_theme(style="whitegrid")

from src.nft_analytics import NFTAnalytics
from src.ethereum_api import EthereumAPI

In [2]:
ggg = NFTAnalytics("0x16780860c1c0A69C9704ad2aC828a92654d7009F")
DATA_FOLDER, RESULT_FOLDER = ggg.make_directories("GreatGenesisGorillas")
TIMESTAMP = datetime.now().strftime("%Y-%m-%d_")

2021-10-23 19:01:12,843 - src.ethereum_api - INFO - Connected to Infura endpoint.


In [3]:
asset_data = ggg.fetch_data(max_offset=2000)
ggg.save_json(asset_data, filename=os.path.join(DATA_FOLDER, "data_gorillas.json"))

2000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [01:25<00:00,  2.10s/it]


2021-10-23 19:02:39,891 - src.nft_analytics - INFO - Saved asset data to data\GreatGenesisGorillas\data_gorillas.json


In [ ]:
asset_data = ggg.load_json(filename=os.path.join(DATA_FOLDER, "data_gorillas.json"))

# Addresses without listing or with above 0.4 ETH listing

In [4]:
listings = {}

for asset in asset_data:
    listings[asset["owner"]["address"]] = []
    
for asset in asset_data:
    if asset["sell_orders"]:
        listing_price = float(asset["sell_orders"][0]["base_price"]) / 1e18
        listings[asset["owner"]["address"]].append(listing_price)
    else:
        listings[asset["owner"]["address"]].append(10000)

In [5]:
valid_addresses = []
min_listing_prices = []

for address, listing_prices in listings.items():
    if min(listing_prices) >= 0.4:
        min_listing_prices.append(min(listing_prices))
        valid_addresses.append(address)

In [6]:
df = pd.DataFrame(columns=["Address", "Minimum Listing"])
df["Address"] = valid_addresses
df["Minimum Listing"] = min_listing_prices

df["Minimum Listing"] = df["Minimum Listing"].replace(10000, 'Unlisted')

opensea_links = []
for address in df["Address"]:
    opensea_links.append(f"https://opensea.io/{address}")
df["OpenSea_link"] = opensea_links

df.to_excel(os.path.join(RESULT_FOLDER, TIMESTAMP + "gorilla_addresses_unlisted_or_aboveequalto0point4ETH.xlsx"), index=False)

In [ ]:
created_dates = []
listing_prices = []
for asset in asset_data:
    if asset["sell_orders"]:
        created_dates.append(parse(asset["sell_orders"][0]["created_date"]))
        listing_prices.append(float(asset["sell_orders"][0]["base_price"]) / 1e18)
listing_prices = np.array(listing_prices)

In [ ]:
len(listing_prices)

In [ ]:
OPENSEA_FLOOR_PRICE = 0.084

In [ ]:
OPENSEA_FLOOR_PRICE = round(OPENSEA_FLOOR_PRICE, 3)

numbers_in_bins = []

price_bins = np.linspace(OPENSEA_FLOOR_PRICE, OPENSEA_FLOOR_PRICE * 2, 9)

last_bin = 0
for current_bin in price_bins:
    number_in_bin = len(listing_prices[(listing_prices < current_bin) & (last_bin < listing_prices)])
    last_bin = current_bin
    numbers_in_bins.append(round(number_in_bin, 2))

price_bins_rounded = np.around(price_bins, 2)

ax = sns.barplot(x=price_bins_rounded, y=numbers_in_bins, color="seagreen")
ax.bar_label(ax.containers[0])
sns.despine(left=True)
ax.set_yticks(ax.get_yticks()[::2])
ax.set_xlabel("Price in ETH")
ax.set_ylabel("Great Genesis Gorillas for sale")
#ax.figure.savefig(os.path.join(RESULT_FOLDER, TIMESTAMP + "great_gorillas.jpg"), dpi=600)

In [ ]:
OPENSEA_FLOOR_PRICE = round(OPENSEA_FLOOR_PRICE, 2)

numbers_in_bins = []

price_bins = np.arange(0.1, 0.6, 0.1)
differential = 0.1
print(price_bins)
print(price_bins - differential)
last_bin = 0
for current_bin in price_bins:
    number_in_bin = len(listing_prices[(listing_prices < current_bin) & (listing_prices >= (current_bin - differential))])
    last_bin = current_bin
    numbers_in_bins.append(round(number_in_bin, 2))

price_bins_rounded = np.around(price_bins, 2)

ax = sns.barplot(x=price_bins_rounded, y=numbers_in_bins, color="seagreen")
ax.bar_label(ax.containers[0])
sns.despine(left=True)
ax.set_yticks(ax.get_yticks()[::2])
ax.set_xlabel("Floor price in ETH")
ax.set_ylabel("Great Gorillas for sale")
ax.set_title("After DELIST")
#ax.figure.savefig(os.path.join(RESULT_FOLDER, TIMESTAMP + "after_delist_great_gorillas.jpg"), dpi=600)